In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel
from transformers import pipeline

In [13]:
#Load model and tokenizer 
model = AutoModelForCausalLM.from_pretrained( "microsoft/Phi-3-mini-4k-instruct", device_map="xpu", torch_dtype="auto", trust_remote_code= True , ) 
# tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini4k-instruct")
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct", trust_remote_code=True)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:12<00:00,  6.25s/it]


In [14]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|>"

In [15]:
# Tokenize the input prompt
inputs_id = tokenizer(prompt, return_tensors="pt").input_ids.to("xpu")

# Generate text
generation_output = model.generate(inputs_id, max_new_tokens=20)

# Print the output
print(tokenizer.decode(generation_output[0]))

You are not running the flash-attention implementation, expect numerical differences.


Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|> Subject: Sincere Apologies for the Gardening Mishap


Dear


In [16]:
print(inputs_id[0].cpu().numpy())

# decode inputs_id
for id in inputs_id[0]:
    print(tokenizer.decode(id), end=" ")

[14350   385  4876 27746  5281   304 19235   363   278 25305   293 16423
   292   286   728   481 29889 12027  7420   920   372  9559 29889 32001]
Write an email apolog izing to Sarah for the trag ic garden ing m ish ap . Exp lain how it happened . <|assistant|> 

In [17]:
print(generation_output[0].cpu().numpy())

print(tokenizer.decode(3323))
print(tokenizer.decode(622))
print(tokenizer.decode([3323, 622]))
print(tokenizer.decode(29901))

[14350   385  4876 27746  5281   304 19235   363   278 25305   293 16423
   292   286   728   481 29889 12027  7420   920   372  9559 29889 32001
  3323   622 29901   317  3742   406  6225 11763   363   278 19906   292
   341   728   481    13    13    13 29928   799]
Sub
ject
Subject
:


In [18]:
text = """ English and CAPITALIZATION show_tokens False None elif == >= else: two tabs:" " Three tabs: "   " 12.0*50=600 """

colors_list = ['102;194;165', '252;141;98', '141;160;203', '231;138;195', '166;216;84', '255;217;47' ]

def show_tokens(sentence, tokenizer_name): 
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name) 
    token_ids = tokenizer(sentence).input_ids 
    for idx, t in enumerate(token_ids): 
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' + 
            tokenizer.decode(t) + 
            '\x1b[0m', end=' ' 
            )

In [19]:
show_tokens(text, "bert-base-uncased")

[CLS] english and capital ##ization show _ token ##s false none eli ##f = = > = else : two tab ##s : " " three tab ##s : " " 12 . 0 * 50 = 600 [SEP] 

In [20]:
show_tokens(text, "bert-base-cased")

[CLS] English and CA ##PI ##TA ##L ##I ##Z ##AT ##ION show _ token ##s F ##als ##e None el ##if = = > = else : two ta ##bs : " " Three ta ##bs : " " 12 . 0 * 50 = 600 [SEP] 

![title](img/2.9.png)

In word2vec, each word is assigned a fixed (static) embedding by simply performing a table lookup. In contrast, when you use a transformer-based model like DeBERTa:

- Static Embedding: The token is first mapped to a static embedding vector.
- Positional Embedding: A positional embedding is then added to encode the token's position in the sequence.
- Transformer Layers: The combined embedding goes through multiple transformer layers where self-attention and feed-forward networks modify it based on the context.

The final output in output.last_hidden_state is a contextualized embedding vector for each token, meaning that the same token can have different representations depending on its surrounding words.

This is a key difference:

- word2vec: Embedding is a simple lookup from a fixed table.
- Transformer models: Embedding is dynamically computed through several processing steps (static embedding, positional encoding, and transformer layers).

Thus, the token representation from a transformer model is much richer and context-sensitive compared to the static lookup in word2vec.

In [21]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base", trust_remote_code=True)
model = AutoModel.from_pretrained("microsoft/deberta-v3-xsmall", trust_remote_code=True)

def analyze_text(input_text, tokenizer, model):
    print(input_text)
    # Break down the text into smaller pieces (tokens) (word, subword, or character)
    tokens = tokenizer(input_text, return_tensors="pt")

    # Pass the tokens to the model
    output = model(**tokens)

    print("Output shape: ", output.last_hidden_state.shape)
    print("Tokens ID: ", tokens['input_ids'])
    print("Decode Tokens:")
    for token in tokens["input_ids"][0]:
        print(token.item(), ":", tokenizer.decode(token), end=" || ")

    print("\n")

# Many modern tokenizers (including those used for models like DeBERTa) are sensitive to whitespace. 
# For example, if a tokenizer uses a byte-level or subword method, it often treats a word differently 
# if it appears at the beginning of a sentence versus when it follows a space.
analyze_text("hello world", tokenizer, model)
analyze_text("hello hello world hello world", tokenizer, model)
analyze_text("hello hello world", tokenizer, model)
analyze_text("world hello world", tokenizer, model)
analyze_text("world hello", tokenizer, model)
analyze_text("Hello World", tokenizer, model)
analyze_text("World Hello", tokenizer, model)

hello world
Output shape:  torch.Size([1, 4, 384])
Tokens ID:  tensor([[    1, 42891,   232,     2]])
Decode Tokens:
1 : [CLS] || 42891 : hello || 232 :  world || 2 : [SEP] || 

hello hello world hello world
Output shape:  torch.Size([1, 7, 384])
Tokens ID:  tensor([[    1, 42891, 20760,   232, 20760,   232,     2]])
Decode Tokens:
1 : [CLS] || 42891 : hello || 20760 :  hello || 232 :  world || 20760 :  hello || 232 :  world || 2 : [SEP] || 

hello hello world
Output shape:  torch.Size([1, 5, 384])
Tokens ID:  tensor([[    1, 42891, 20760,   232,     2]])
Decode Tokens:
1 : [CLS] || 42891 : hello || 20760 :  hello || 232 :  world || 2 : [SEP] || 

world hello world
Output shape:  torch.Size([1, 5, 384])
Tokens ID:  tensor([[    1,  8331, 20760,   232,     2]])
Decode Tokens:
1 : [CLS] || 8331 : world || 20760 :  hello || 232 :  world || 2 : [SEP] || 

world hello
Output shape:  torch.Size([1, 4, 384])
Tokens ID:  tensor([[    1,  8331, 20760,     2]])
Decode Tokens:
1 : [CLS] || 8331 :

In [22]:
from sentence_transformers import SentenceTransformer 

# Load model 
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2") 
# Convert text to text embeddings 
vector = model.encode("Best movie ever!")

print(vector.shape)

(768,)


# Training a Song Embedding Model

In [ ]:
import pandas as pd
from urllib import request



In [39]:
# Get the playlist dataset file
data = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt')

# Parse the playlist dataset file. Skip the first two lines as
# they only contain metadata
lines = data.read().decode("utf-8").split('\n')[2:]

# Remove playlists with only one song
playlists = [s.rstrip().split() for s in lines if len(s.split())> 1]


In [43]:
# Load song metadata
songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')
songs_file = songs_file.read().decode("utf-8").split('\n')
songs = [s.rstrip().split('\t') for s in songs_file]
songs_df = pd.DataFrame(data=songs, columns = ['id', 'title', 'artist'])
songs_df = songs_df.set_index('id')

In [66]:
print( 'Playlist #1:\n ', playlists[0], '\n')
print( 'Playlist #2:\n ', playlists[1])

# print(songs_df.head())
# print(songs_df.iloc[0]['title'])
# print(type(int(playlists[0][0])))
# print(songs_df.iloc[int(playlists[0][0])])

for i in range(3):
    print('Song in playlist ', i, ': ')
    for j in range(5):
        print(songs_df.iloc[int(playlists[i][j])]['title'], end=" | ")
    print(" ...")

Playlist #1:
  ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '2', '42', '43', '44', '45', '46', '47', '48', '20', '49', '8', '50', '51', '52', '53', '54', '55', '56', '57', '25', '58', '59', '60', '61', '62', '3', '63', '64', '65', '66', '46', '47', '67', '2', '48', '68', '69', '70', '57', '50', '71', '72', '53', '73', '25', '74', '59', '20', '46', '75', '76', '77', '59', '20', '43'] 

Playlist #2:
  ['78', '79', '80', '3', '62', '81', '14', '82', '48', '83', '84', '17', '85', '86', '87', '88', '74', '89', '90', '91', '4', '73', '62', '92', '17', '53', '59', '93', '94', '51', '50', '27', '95', '48', '96', '97', '98', '99', '100', '57', '101', '102', '25', '103', '3', '104', '105', '106', '107', '47', '108', '109', '110', '111', '112', '113', '25', '63', '62', '114', '115', '84', '116', '117',

In [68]:
from gensim.models import Word2Vec

# Train our Word2Vec model
model = Word2Vec(
    playlists, vector_size=32, window=20, negative=50, min_count=1, workers=4
)

In [69]:
song_id = 2172
# Ask the model for songs similar to song #2172
model.wv.most_similar(positive=str(song_id))

[('3119', 0.9978693723678589),
 ('2849', 0.9977297186851501),
 ('3126', 0.9970256686210632),
 ('11473', 0.9968244433403015),
 ('6626', 0.9958199262619019),
 ('10105', 0.9957699179649353),
 ('6658', 0.9956603050231934),
 ('1922', 0.9956520199775696),
 ('9995', 0.9956437945365906),
 ('10084', 0.9954245686531067)]

In [83]:
import numpy as np

def similar_song(song_id):
    results = np.array(model.wv.most_similar(positive=str(song_id), topn=5))[:,0]
    print("Similar song with: ", songs_df.iloc[int(song_id)]["title"], " by: ", songs_df.iloc[int(song_id)]["artist"])
    print(songs_df.iloc[results])

similar_song(42231)

Similar song with:  I Let A Song Go Out Of My Heart  by:  Grant Stewart
                        title                       artist
id                                                        
36497         Never Let Me Go                  Stacey Kent
54919             Pharoah Joy  Joe Locke & David Hazeltine
56020   Afro 6\/8 Minor Blues              Larry Vuckovich
36507              FluteVibes               Gerald Beckett
44093            Another Star                 Cedar Walton
